In [1]:
import tkinter as tk
import requests
import json
import tkinter.messagebox as tkmb
from datetime import datetime
from operator import itemgetter
import socketio
import threading 

In [2]:
root = tk.Tk()
window1 = tk.Toplevel(root)
#window2 = tk.Toplevel(root)

In [3]:

name = tk.StringVar()
contact = tk.StringVar()
password = tk.StringVar()
confirm_password = tk.StringVar()


add_phone_num2 = tk.StringVar()
add_friend_name = tk.StringVar()


name_lbl = tk.Label(master=window1, text='Username')
contct_lbl  = tk.Label(master=window1, text="Contact")
pwd_lbl = tk.Label (master=window1,text="Password")
#cpwd_lbl = tk.Label(text="Confirm Password")


name_lbl.grid(row=0,column=0,sticky='w')    
contct_lbl.grid(row=1,column=0,sticky='w')  
pwd_lbl.grid(row=2,column=0,sticky='w')  
#cpwd_lbl .grid(row=3,column=0,sticky='w') 

name_txt   = tk.Entry(master=window1,textvariable=name)
contct_txt = tk.Entry(master=window1,textvariable=contact)
pwd_txt    = tk.Entry(master=window1,textvariable=password,show='*')
#cpwd_txt   = tk.Entry(textvariable=confirm_password)

sio = socketio.Client()

def addFriendClicked():
    
    url = 'http://127.0.0.1:5000/chatwindow/'
    
    phone_num1  = contact.get()
    friend_name = add_friend_name.get()
    phone_num2  = add_phone_num2.get()
    
    excluded_char = '`~!@#$%^&*()'
    special_char = False
    for x in excluded_char:
              if x in friend_name:
                 special_char = True
    
    if ( (friend_name == '') or 
         (4 >= len(friend_name)) or 
         (20 < len(friend_name)) ):
           
              tkmb.showerror ('Error', message='Name length must be between 5 and 20 characters')
    elif special_char :
          
                 tkmb.showerror ('Error', message='Special characters are not allowed')
        
    elif ( (phone_num2 == '')  or (phone_num2.isdigit() != True) or (len(phone_num2) != 10) ) :
              tkmb.showerror ('Error', message='Contact number should have 10 digits with no spaces in between')    
          
    
    else:
         query_strings = {'phone_num1': phone_num1, 'phone_num2':phone_num2,'friend_name':friend_name}
    
         response2 = requests.post(url,params = query_strings)
         print (response2.status_code)
         print (response2.headers)
         print (response2.content)
         print (response2.text)
         
    if response2.status_code == 200:   
        tkmb.showinfo('Success', message=response2.text)
        
    else:    
        tkmb.showinfo('Information', message=response2.text)
        




def create_new_window():
    window1.withdraw()
    window2 = tk.Toplevel(root)
    
    friend_name_lbl = tk.Label(master=window2,text="Contact Name")
    friend_name_txt = tk.Entry(master=window2,textvariable = add_friend_name)
    
    friend_num_lbl = tk.Label(master=window2,text="Contact Number")
    friend_num_txt = tk.Entry(master=window2,textvariable = add_phone_num2)
    
    add_friend_btn = tk.Button(master=window2,text="Add Friend",
                               width=5,
                               height=2,
                               bg="blue",
                               fg="black",
                               command=addFriendClicked)
    
    
    friend_num_lbl.grid(row=0,column=0)
    friend_num_txt.grid(row=0,column=1)
    
    friend_name_lbl.grid(row=1,column=0)
    friend_name_txt.grid(row=1,column=1)
    
    add_friend_btn.place(x=100,y=80)
    
#########################################################

    def display_msg_log(phone_num1,phone_num2):
        
        
        url = 'http://127.0.0.1:5000/msglog/'
        query_strings = {'P1': phone_num1, 'P2':phone_num2}
    
        response5 = requests.get(url,params = query_strings)
        print (response5.status_code)
        print (response5.headers)
        print (response5.content)
        print ('response5.text: ', response5.text)
        #message_log = json.loads(response5.text) 
        print (type(json.loads(response5.text) ))
        
        return (json.loads(response5.text) )
        
    
#########################################################

#########################################################

    
        
#########################################################


#########################################################

    def choose_friend(event):
        
        
        def connect_to_server():
        #sio = socketio.Client()
            instant_msg = True
               
            @sio.on('connect')
            def on_connect():
                      print('Client connected to server')
     
            @sio.on('assign_client_id')     
            def on_assign_client_id(client_id):
                       print ('My client id is: ',client_id)
                
            @sio.on('latest_msg') 
            def on_latest_msg(chat_details):
                       print ('on_latest_msg function')
                       print ('Message from friend: ',json.loads(chat_details))
                       message_log = json.loads(chat_details)
                       date_format = '%a, %d %b %Y %H:%M:%S %Z'
                       compare_dt = datetime.min.date()
              
                       for item in message_log:
                                    print ('item: ', item)
                                    dt = datetime.strptime(item[6], date_format)
                                    print ('dt == compare_dt: ',dt.date() == compare_dt)
                                #print ('type: ',type(dt),type(compare_dt))
                                    print ('dates:',compare_dt,dt.date())
                                    if (dt.date() != compare_dt ) :
                                                        compare_dt = dt.date()
                                                        print ('dates inside if loop: ',compare_dt,dt.date())
                                                        date_lbl = tk.Label(master=window3,text=dt.strftime("%d/%m/%Y"))
                                                        date_lbl.pack()
   
                                                        sender_lbl  = tk.Label(master=window3,text=(item[2]+': '))
                                                        message_lbl = tk.Label(master=window3,text=item[5])
                                                        time_lbl    = tk.Label(master=window3,text=dt.strftime('%H:%M'))
                
                                    sender_lbl.pack()
                                    message_lbl.pack()
                                    time_lbl.pack()
                            
                       print ("on_latest_msg exit")            
            
            @sio.on('errors')    
            def on_errors(error_details):
                  print ('Error details: ', error_details)
                
            sio.connect('http://localhost:5000')
            sio.wait()    
        
        def send_message(phone_num1,phone_num2,selected_friend,message_txt,instant_msg,message_log=[]):
                    instant_msg = True
                    if len( message_txt.get(1.0, "end-1c")) == 0:
                          print ('Write a msg to your friend')
                    else:
                            print ('message: ',  message_txt.get(1.0, "end-1c"))
                            receiver    = selected_friend
                            sender      = name.get()
                            input_txt_msg = message_txt.get(1.0, "end-1c")
                            input_details = {'P1': phone_num1, 'sender':sender,'P2':phone_num2,'receiver':receiver,'message': input_txt_msg}
            
                            print ('sio.connected: ',sio.connected)
                            if sio.connected == True:
                                print ('sending message to friend: ', receiver,input_txt_msg)
                                sio.emit('msg_to_frnd', data=input_details)
                
                                @sio.on('chat_table_update_resp')    
                                def on_chat_table_update_resp(tab_upd_resp):
                                        print ('Chat table update response: ', tab_upd_resp)
                      
                                @sio.on('latest_msg')
                                def on_latest_msg(chat_details):
                                    print ('Latest Message from friend: ',json.loads(chat_details))
                                    date_format = '%a, %d %b %Y %H:%M:%S %Z'
                                    compare_dt = datetime.min.date()
                                    message_log = json.loads(chat_details)
                                    for item in message_log:
                                                print ('item: ', item)
                                                dt = datetime.strptime(item[6], date_format)
                                                print ('dt == compare_dt: ',dt.date() == compare_dt)
                                #print ('type: ',type(dt),type(compare_dt))
                                                print ('dates:',compare_dt,dt.date())
                                                if (dt.date() != compare_dt ) :
                                                        compare_dt = dt.date()
                                                        print ('dates inside if loop: ',compare_dt,dt.date())
                                                        date_lbl = tk.Label(master=window3,text=dt.strftime("%d/%m/%Y"))
                                                        date_lbl.pack()
   
                                                        sender_lbl  = tk.Label(master=window3,text=(item[2]+': '))
                                                        message_lbl = tk.Label(master=window3,text=item[5])
                                                        time_lbl    = tk.Label(master=window3,text=dt.strftime('%H:%M'))
                
                                                sender_lbl.pack()
                                                message_lbl.pack()
                                                time_lbl.pack()
                            #print ("on_latest_msg exit")  
                            #return (message_log)
                            
                                @sio.on('errors')    
                                def on_errors(error_details):
                                        print ('Error details: ', error_details)       
                
                            else:    
            
            #sio = socketio.Client()              
                                @sio.on('connect')
                                def on_connect():
                                    print('Client connected to server')
        
                                @sio.on('assign_client_id')     
                                def on_assign_client_id(client_id):
                                      print ('My client id is: ',client_id)
                #sio.emit('recv_msg', data=input_details)
                        
                                @sio.on('chat_table_update_resp')    
                                def on_chat_table_update_resp(tab_upd_resp):
                                      print ('Chat table update response: ', tab_upd_resp)
                                
                                @sio.on('latest_msg') 
                                def on_latest_msg_from_frnd(chat_details):
                                            print ('on_latest_msg_from_frnd function')
                                            print ('Message from friend: ',json.loads(chat_details))
                                            message_log = json.loads(chat_details)
                    
                                            print ("on_latest_msg_from_frnd exit") 
                    #return (message_log)
          
                                @sio.on('errors')    
                                def on_errors(error_details):
                                           print ('Error details: ', error_details)
                
                                sio.connect('http://localhost:5000')
                                sio.wait()
                                print ('exiting send_message function')

        
        instant_msg = False
        global message_log 
        
        #sio = socketio.Client()
                
        t1 = threading.Thread(target=connect_to_server)
        t1.start()

        selected_friend = friend_list_txt.get('current linestart', 'current lineend')
        print (selected_friend)
        
        window3 = tk.Toplevel(root)
        
        phone_num1 = contact.get()
        for x in range(len(friend_list)):
                   if friend_list[x][0] == selected_friend:
                        phone_num2 = friend_list[x][1]

 
        display_name = tk.Label(master=window3,width=10, height=1, text=selected_friend) 
        display_name.pack()
        
        message_log = display_msg_log(phone_num1,phone_num2)
        
        date_format = '%a, %d %b %Y %H:%M:%S %Z'
        compare_dt = datetime.min.date()
        
        for item in message_log:
                #print ('item: ', item)
                dt = datetime.strptime(item[6], date_format)
                #print ('dt == compare_dt: ',dt.date() == compare_dt)
                #print ('type: ',type(dt),type(compare_dt))
                #print ('dates:',compare_dt,dt.date())
                if (dt.date() != compare_dt ) :
                            compare_dt = dt.date()
                            #print ('dates inside if loop: ',compare_dt,dt.date())
                            date_lbl = tk.Label(master=window3,text=dt.strftime("%d/%m/%Y"))
                            date_lbl.pack()
   
                sender_lbl  = tk.Label(master=window3,text=(item[2]+': '))
                message_lbl = tk.Label(master=window3,text=item[5])
                time_lbl    = tk.Label(master=window3,text=dt.strftime('%H:%M'))
                
                sender_lbl.pack()
                message_lbl.pack()
                time_lbl.pack()

        message_txt = tk.Text(master=window3,width=20,height=3)

        t2 = threading.Thread(target=send_message, args = (phone_num1,phone_num2,selected_friend,message_txt,instant_msg))

        send_btn  = tk.Button(master=window3,text="Send",command=lambda: (threading.Thread(target=send_message, args = (phone_num1,phone_num2,selected_friend,message_txt,instant_msg))
).start())
        
        print ('instant_msg before : ', instant_msg)
        
        try:
                      t2.join()
                      if instant_msg:
                                print ('instant_msg after : ',instant_msg)
                                print ('message_log: ',message_log)
                                for item in message_log:
                                            print ('item: ', item)
                                            dt = datetime.strptime(item[6], date_format)
                                            print ('dt == compare_dt: ',dt.date() == compare_dt)
                                #print ('type: ',type(dt),type(compare_dt))
                                            print ('dates:',compare_dt,dt.date())
                                            if (dt.date() != compare_dt ) :
                                                        compare_dt = dt.date()
                                                        print ('dates inside if loop: ',compare_dt,dt.date())
                                                        date_lbl = tk.Label(master=window3,text=dt.strftime("%d/%m/%Y"))
                                                        date_lbl.pack()
   
                                                        sender_lbl  = tk.Label(master=window3,text=(item[2]+': '))
                                                        message_lbl = tk.Label(master=window3,text=item[5])
                                                        time_lbl    = tk.Label(master=window3,text=dt.strftime('%H:%M'))
                
                                            sender_lbl.pack()
                                            message_lbl.pack()
                                            time_lbl.pack()
        except Exception as e:
                      print("Exception Handled in Main, Detials of the Exception:", e)
        
        
        
 

        #print (input_txt_msg)
        message_txt.pack()
        send_btn.pack()
        print ("window3.bindtags(): ",window3.bindtags())
        


##########################################################


    url = 'http://127.0.0.1:5000/displaylist/'
    
    query_strings = {'phone_num1': contact.get()}
    
    response3 = requests.get(url,params = query_strings)
    
    print ('response3.status_code: ', response3.status_code)
    #print (response3.content)
    print (response3.text)
   
    friend_list = json.loads(response3.text)    
    print ('friend_list',friend_list)
    print (type(friend_list))
    if response3.status_code == 200:
        i = 0
        height = len(friend_list)
        friend_list_txt = tk.Text(master=window2, width=10, height=height, fg='blue') 
        friend_list_txt.grid(row=10, column=0)
       
        for j in range(len(friend_list)):
            print ('j: ',j)
            #i = j+10
            
            friend_list_txt['state'] = tk.NORMAL
            friend_list_txt.insert(tk.END, friend_list[j][0] + "\n")
            friend_list_txt['state'] = tk.DISABLED
            
                  
   #         i= i+1
    print ("window2.bindtags(): ",window2.bindtags())
    friend_list_txt.bind('<Button-1>', choose_friend)

        
    
    #selected_friend = friend_list_txt.index()
    #friend_list_txt.bind('<Button-1>', send_message)    
    


def submitClicked():
    url = 'http://127.0.0.1:5000/signin/'
    #url = 'https://api.github.com'
    username   = name.get()
    contact_no = contact.get()
    pwd        = password.get()
    
    excluded_char = '`~!@#$%^&*()'
    special_char = False
    for x in excluded_char:
              if x in username:
                 special_char = True
    
    if ( (username == '') or (4 >= len(username)) or (64 < len(username)) ):
              tkmb.showerror ('Error', message='Name length must be between 5 and 64 characters')
    elif special_char:
          
                 tkmb.showerror ('Error', message='Special characters are not allowed')
                
    elif ( (contact_no == '')  or (contact_no.isdigit() != True) or (len(contact_no) != 10) ) :
              tkmb.showerror ('Error', message='Contact number should have 10 digits with no spaces in between')
    elif ( (pwd == '') or (len(pwd) != 8)):
            tkmb.showerror ('Error', message='Password should be at least 8 characters long')
    else:        
        
    
             query_strings = {'username' : username,'contact': contact_no, 'password':pwd}
    #query_strings = {'username' : username}
    #response = requests.get(url,params = query_strings)
    #response = requests.get(url)
             response = requests.get(url,params = query_strings)
             print (response.status_code)
             print (response.headers)
             print (response.content)
             print (response.text)
             #print (dir(response))
             if response.status_code == 200:           
                        tkmb.showinfo('Success', message=response.text)
                        create_new_window() 
             else:
                        tkmb.showerror ('Error',message=response.text)

      
    

                    
name_txt.grid(row=0,column=2)    
contct_txt.grid(row=1,column=2)   
pwd_txt.grid(row=2,column=2) 
#cpwd_txt.grid(row=3,column=2)  


signin_btn = tk.Button(master=window1,
    text="Sign in",
    width=5,
    height=2,
    bg="blue",
    fg="black",
    command= submitClicked
)    

signin_btn.place(x=100,y=100)


newuser_lbl = tk.Label (master=window1,text="New User?")
newuser_lbl.place(x=105,y=150)


In [ ]:
root.mainloop()

200
{'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '6', 'Date': 'Mon, 11 Oct 2021 05:19:44 GMT'}
b'All OK'
All OK
response3.status_code:  200
[["Aishu", "1234567890"], ["NAjay", "1234567899"], ["NSivaP", "7702954705"]]
friend_list [['Aishu', '1234567890'], ['NAjay', '1234567899'], ['NSivaP', '7702954705']]
<class 'list'>
j:  0
j:  1
j:  2
window2.bindtags():  ('.!toplevel2', 'Toplevel', 'all')
NSivaP
200
{'Content-Type': 'application/json', 'Content-Length': '319', 'Date': 'Mon, 11 Oct 2021 05:19:49 GMT'}
b'[[3, "9704056271", "Susmitha", "7702954705", "NSivaP", "Hi Siva", "Wed, 22 Sep 2021 12:01:35 GMT"], [11, "9704056271", "Susmitha", "7702954705", "NSivaP", "Are you in office?", "Wed, 22 Sep 2021 17:41:06 GMT"], [104, "7702954705", "NSivaP", "9704056271", "HomeMinister", "siva prasad", "Mon, 11 Oct 2021 10:42:47 GMT"]]'
response5.text:  [[3, "9704056271", "Susmitha", "7702954705", "NSivaP", "Hi Siva", "Wed, 22 Sep 2021 12:01:35 GMT"], [11, "9704056271", "Susmitha", "7